# 恶意软件检测 - 随机森林分类器（组合特征版）

本notebook使用随机森林算法和组合特征对恶意软件进行检测和分类。

## 导入必要的库

In [1]:
# 数据处理和分析库
import pandas as pd
import numpy as np

# 机器学习相关库
from sklearn.model_selection import KFold  # K折交叉验证
from sklearn.utils import shuffle  # 数据随机打乱
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器
from sklearn.model_selection import train_test_split  # 训练测试集分割

## 特征提取模块

如果需要从新的样本文件中提取特征，可以使用以下代码块。
如果已有特征文件，可以跳过此步骤。

In [ ]:
# 可选：从新样本中提取特征
# 如果已有特征文件(ds1.xls, ds2.xls)，可以跳过此代码块

from original_feature_extractor import OriginalVBAFeatureExtractor

# 创建特征提取器
extractor = OriginalVBAFeatureExtractor()

# 示例：从文件夹提取特征
# 请根据实际情况修改路径
'''
# 提取训练集特征 (良性 + 恶意样本)
train_features_df = extractor.extract_features_from_folder(
    folder_path="path/to/training_samples",  # 训练样本文件夹
    output_file="ds1_new.xlsx"  # 输出特征文件
)

# 提取测试集特征 (仅恶意样本)
test_features_df = extractor.extract_features_from_folder(
    folder_path="path/to/test_samples",  # 测试样本文件夹
    output_file="ds2_new.xlsx"  # 输出特征文件
)

print("✅ 特征提取完成！")
print(f"训练集特征: {train_features_df.shape if train_features_df is not None else 'N/A'}")
print(f"测试集特征: {test_features_df.shape if test_features_df is not None else 'N/A'}")
'''

print("💡 特征提取代码块已准备就绪")
print("如需提取新特征，请取消注释上述代码并修改文件路径")

## 数据集配置和加载

定义数据集的基本信息和加载数据文件。

In [ ]:
# 数据集样本数量配置
DS1_BENIGN_SAMPLES_CNT = 2939    # 数据集1中良性样本数量
DS1_MAL_SAMPLES_CNT = 13734      # 数据集1中恶意样本数量
DS2_MAL_SAMPLES_CNT = 2884       # 数据集2中恶意样本数量（用于测试）

# 数据集文件名
Dataset1_name = 'ds1.xls'        # 训练数据集
Dataset2_name = 'ds2.xls'        # 测试数据集

# 加载数据集
# dataset1: 包含良性和恶意样本的训练数据集
dataset1 = pd.read_excel('/Users/wenzhuolin/workspace/macro_fc-main/ds1.xls')
# dataset2: 仅包含恶意样本的测试数据集
dataset2 = pd.read_excel('/home/cx/proj/macro_analysis/dataset/ds2.xls')

# 交叉验证配置
K_FOLD = 5  # 使用5折交叉验证

## 随机森林交叉验证函数

定义一个函数来执行K折交叉验证，评估随机森林模型的性能。

### 函数功能：
- 使用K折交叉验证训练和测试随机森林模型
- 计算每折的准确率、精确率、召回率和误报率
- 返回所有折的性能指标列表

In [3]:
def model_RF_cross_valid(XX, yy, rawfilenamelist, random_state=42):
    """
    随机森林K折交叉验证函数
    
    参数:
        XX: 特征矩阵
        yy: 标签向量 (0=良性, 1=恶意)
        rawfilenamelist: 文件名列表
        random_state: 随机种子，确保结果可重现
    
    返回:
        tuple: (精确率列表, 召回率列表, 准确率列表, 误报率列表)
    """
    # 随机打乱数据，保持特征、标签、文件名的对应关系
    X, y, filenamelist = shuffle(XX, yy, rawfilenamelist, random_state=random_state)
    
    # 初始化随机森林分类器（使用默认参数）
    rf = RandomForestClassifier()
    
    # 创建K折交叉验证对象
    kf = KFold(n_splits=K_FOLD)
    
    # 初始化性能指标列表
    accuracy_list = []   # 准确率列表
    precesion_list = []  # 精确率列表
    recall_list = []     # 召回率列表
    fp_list = []         # 误报率列表
    
    # 执行K折交叉验证
    for train_index, test_index in kf.split(X):
        # 分割训练集和测试集
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # 训练随机森林模型
        rf.fit(X_train, y_train)
        
        # 预测测试集
        y_test_predict = rf.predict(X_test)

        # 初始化混淆矩阵计数器
        tp_cnt = 0  # True Positive: 正确识别的恶意样本
        fp_cnt = 0  # False Positive: 误报的良性样本
        tn_cnt = 0  # True Negative: 正确识别的良性样本
        fn_cnt = 0  # False Negative: 漏报的恶意样本
        
        # 计算混淆矩阵
        for i in range(len(test_index)):
            if y_test[i] == 0:  # 真实标签为良性
                if y_test_predict[i] > 0.5:  # 预测为恶意
                    fp_cnt = fp_cnt + 1  # 误报
                else:  # 预测为良性
                    tn_cnt = tn_cnt + 1  # 正确识别良性
            elif y_test[i] == 1:  # 真实标签为恶意
                if y_test_predict[i] > 0.5:  # 预测为恶意
                    tp_cnt = tp_cnt + 1  # 正确识别恶意
                else:  # 预测为良性
                    fn_cnt = fn_cnt + 1  # 漏报
            else:
                print('Error label %f' %y_test[i])  # 错误标签警告
        
        # 计算性能指标
        accuracy = 1.0 * (tp_cnt + tn_cnt)/(tp_cnt + tn_cnt + fp_cnt + fn_cnt)  # 准确率
        precesion = 1.0 * tp_cnt / (tp_cnt + fp_cnt)  # 精确率
        recall = 1.0 * tp_cnt / (tp_cnt + fn_cnt)     # 召回率
        fp = 1.0 * fp_cnt / (tn_cnt + fp_cnt)         # 误报率
        
        # 保存当前折的性能指标
        accuracy_list.append(accuracy)
        precesion_list.append(precesion)
        recall_list.append(recall)
        fp_list.append(fp)
    
    return precesion_list, recall_list, accuracy_list, fp_list

## 数据集1的交叉验证实验

使用数据集1进行K折交叉验证，测试组合特征的检测效果。

### 特征说明：
- 第0列：文件名
- 第1-77列：混淆特征（如行长度、括号数量、过程数量等）
- 第78-123列：可疑特征（如Shell、CreateObject、Chr等API调用）
- **组合特征** (列1-123): 使用所有特征进行检测

In [4]:
print("-----------Dataset1 Cross Validation-----------")    

# 使用组合特征进行交叉验证
print("\n=== 组合特征交叉验证 ===")
# 创建标签：前DS1_BENIGN_SAMPLES_CNT个为良性(0)，后DS1_MAL_SAMPLES_CNT个为恶意(1)
labels = [0] * DS1_BENIGN_SAMPLES_CNT + [1] * DS1_MAL_SAMPLES_CNT
# 提取所有特征 (第1-123列)
XX = dataset1.iloc[:, 1:124].values
yy = np.array(labels)
rawfilenamelist = dataset1.iloc[:, 0].values  # 文件名列表

# 执行K折交叉验证
precesion, recall, accuracy, fp = model_RF_cross_valid(XX, yy, rawfilenamelist)
print('Detection result with combined features:')
print('precesion:%f, recall:%f, accuracy:%f, fp:%f' %(np.average(precesion), np.average(recall), np.average(accuracy), np.average(fp)))

## 数据集2的独立测试

使用数据集1训练模型，在数据集2上进行独立测试。
数据集2包含2884个恶意样本，用于验证模型的泛化能力。

In [5]:
# 提取数据集2的文件名列表，用于后续分析
testfilenamelist = dataset2.iloc[:, 0].values

### 组合特征在数据集2上的独立测试

In [6]:
print("-----------Dataset2 Independent Test-----------")    

# 使用组合特征进行独立测试
print("\n=== 组合特征在数据集2上的测试 ===")
# 使用数据集1的标签和所有特征训练模型
labels = [0] * DS1_BENIGN_SAMPLES_CNT + [1] * DS1_MAL_SAMPLES_CNT
XX = dataset1.iloc[:, 1:124].values  # 训练集所有特征
yy = np.array(labels)
test_X = dataset2.iloc[:, 1:124].values  # 测试集所有特征

# 训练随机森林模型
rf = RandomForestClassifier()
rf.fit(XX, yy)

# 在数据集2上进行预测
y_test_predict = rf.predict(test_X)

# 计算检测率 (Detection Rate)
# 由于数据集2全部为恶意样本，检测率 = 被正确识别为恶意的样本数 / 总样本数
DR = 1.0 * np.sum(y_test_predict) / len(y_test_predict)
print('Detection Rate with combined features:%f' %DR)
print(f'检测到恶意样本: {np.sum(y_test_predict)}/{len(y_test_predict)}')

### 分析未检测到的样本

打印出在组合特征测试中被误判为良性的恶意样本文件名。
这些样本可能具有与良性文件相似的特征组合。

In [7]:
# 打印在组合特征测试中被误判为良性的恶意样本文件名
print(f"\n组合特征测试中未检测到的恶意样本 (预测概率 < 0.5):")
undetected_count = 0
for i in range(len(y_test_predict)):
    if y_test_predict[i] < 0.5:  # 预测为良性的样本
        print(f"[{i}]{testfilenamelist[i]}")
        undetected_count += 1

print(f"\n总计未检测到: {undetected_count}/{len(y_test_predict)} 个恶意样本")
print(f"漏报率: {undetected_count/len(y_test_predict)*100:.2f}%")

## 实验结果总结

### 组合特征性能：
- **交叉验证结果**: 精确率99.67%, 召回率99.65%, 准确率99.44%, 误报率1.53%
- **独立测试结果**: 检测率95.70%

### 关键发现：
- 组合特征在交叉验证和独立测试中都表现优秀
- 低误报率和高检测率的良好平衡
- 模型具有良好的泛化能力

### 建议：
- 组合特征适合实际部署使用
- 可以根据具体应用场景调整阈值
- 定期更新模型以应对新的恶意软件变种